## Convoluciones y Max Pooling

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import misc

### Cargamos Imagen de Scipy

Para ejemplificar la razón en el uso de las capas convolucionales y de max-pooling, vamos a implementar un pequeño ejemplo, donde utilizaremos una imagen de prueba de de Scipy para ver el efecto que se tiene sobre la imagen, después se haber aplicado las capas.

<img src="img/c1.png" />

Por ejemplo, si tenemos una imagen de una “X” al pasar la imagen a blanco y negro, podemos hacer una representación del color negro como -1 y el blanco como +1. La imagen anterior representa esta conversión inicial. 

Ahora, vamos a usar una imagen real en blanco y negro de tamaño 512x512 y la vamos a desplegar en pantalla:

In [ ]:
# desplegamos una imagen demo de Scipy
i = misc.ascent()
plt.grid(False)
plt.gray()
plt.imshow(i)
plt.show()

print(i.shape)

### Creamos una Copia y un Filtro (3x3)

El filtro es el patrón (normalmente una matriz de 3*3) que le ayuda a la capa convolucional a enfocarse en ciertos aspectos de la imagen. Por tanto, si se cambia el filtro, la convolución causa que la imagen resalte otros atributos. 

<img src="img/c2.png" />

Para ejemplificar esto, utilizando el ejemplo de la "X" vamos identificar 3 posibles filtros que se pueden utilizar para extraer atributos de la imagen. La operación convolucional, es aquella donde se aplica el filtro a una imagen de n x n y se devuelve otra imagen de n x n pero transformada por el filtro.

En el siguiente código, tenemos varios filtros declarados, solamente vamos a usar uno de 3x3. Ud puede cambiar el filtro si gusta. También vamos a hacer una copia de la imagen en blanco y negro para ver efecto del filtro sobre la imagen original.

In [ ]:
# copia de la imagen y dimensiones
i_transformed = np.copy(i)
size_x = i_transformed.shape[0]
size_y = i_transformed.shape[1]

# creamos un filtro (puede probar con varios)
filter = [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]
# filter = [[0, 1, 0], [1, -4, 1], [0, 1, 0]]
# filter = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]

print(filter)

### Capa Convolucional

<img src="img/c3.png" />

Utilizando el ejemplo de la “X” vamos a aplicar el “filtro” seleccionado a través de toda la imagen original. La operación del filtro se realiza de la siguiente forma:

- Se inicia en la posición 0,0 de la imagen original. 
- Se trabaja con las posiciones de la imagen que son homologas al filtro.
- Cada pixel del filtro, se multiplica por su equivalente en la posición de la imagen. 
- Se suman todas las multiplicaciones y se genera un promedio. 
- El promedio corresponde a la posición 0,0 de una nueva matriz.  
- Ahora nos movemos a la posicion 0,1 de la imagen, y repetimos todo el proceso hasta abarcar toda la imagen i x j. 


In [ ]:
weight  = 1

for x in range(1,size_x-1):
    for y in range(1,size_y-1):
        convolution = 0.0
        convolution = convolution + (i[x - 1, y-1] * filter[0][0])
        convolution = convolution + (i[x, y-1] * filter[0][1])
        convolution = convolution + (i[x + 1, y-1] * filter[0][2])
        convolution = convolution + (i[x-1, y] * filter[1][0])
        convolution = convolution + (i[x, y] * filter[1][1])
        convolution = convolution + (i[x+1, y] * filter[1][2])
        convolution = convolution + (i[x-1, y+1] * filter[2][0])
        convolution = convolution + (i[x, y+1] * filter[2][1])
        convolution = convolution + (i[x+1, y+1] * filter[2][2])
        convolution = convolution * weight
        if(convolution<0):
            convolution=0
        if(convolution>255):
            convolution=255
        i_transformed[x, y] = convolution

In [ ]:
plt.gray()
plt.grid(False)
plt.imshow(i_transformed)
plt.show()   

print(i_transformed.shape)

### Capa de Max-Pooling

La capa de max-pooling es una capa de reducción de dimensionalidad. La idea es reducir el tamaño de la imagen a la mitad, pero conservando los features lo mejor que se pueda.

La capa de pooling usualmente se ejecuta después de una capa convolucional, para reducir el tamaño de la imagen, a tal punto de que cuando la imagen resultante llegue a la capa flatten de MLP (multilayer perceptron), la imagen que paso por las capas convolucionales y de max pooling conserven los features mas importantes, pero con un tamaño de imagen pequeño que acelera y mejora la calidad de las predicciones de cualquier tipo de datos que venga en forma de matriz.

<img src="img/c4.png">

La idea del max-pooling (2,2) es que de una matriz de tamaño 2x2, se va a seleccionar el valor mas grande de la imagen resultante de la capa convolucional para generar una nueva matriz reducida pero representativa de los features seleccionados por el filtro en la capa convolucional.

<img src="img/c5.png">


In [ ]:
new_x = int(size_x/2)
new_y = int(size_y/2)
newImage = np.zeros((new_x, new_y))
for x in range(0, size_x, 2):
    for y in range(0, size_y, 2):
        pixels = []
        pixels.append(i_transformed[x, y])
        pixels.append(i_transformed[x+1, y])
        pixels.append(i_transformed[x, y+1])
        pixels.append(i_transformed[x+1, y+1])
        newImage[int(x/2),int(y/2)] = max(pixels)

In [ ]:
plt.gray()
plt.grid(False)
plt.imshow(newImage)
plt.show()   

print(newImage.shape)